# World Development Indicators (1960 - 2014): Data Cleaning

## Objectives

* Assess the database structure, including tables and columns relevant for analysis.

* Clean the data (i.e. ensure correct data formatting, fill/remove blank fields, find and remove duplicate records)

## Methodology

* Apply SQLite3 connection and queries to probe database

* Store query results within tuples (for individual results) or pandas DataFrames (for tabular results)

## Results

* Two of the six tables are neceassary for analysis (*Country* and *Indicators*), although the *Series* table can provide indicator definitions if needed.

* All data was already correctly formatted.

* Of the two relevant tables, blank fields were only present in the *Country* table and were filled appropriately, depending on the feature column.

* There were no duplicate records in the two relevant tables.

### Install relevant packages and connect to database

In [1]:
import pandas as pd
import sqlite3
import numpy as np

# Connect to the database or create one if it doesn't exist
connection = sqlite3.connect('indicators.sqlite')

# Create a cursor object to interact with the database
cursor = connection.cursor()


### Output table and column names

In [2]:
# Execute a query to get all table names from the sqlite_master table
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch the results (table names)
tables = cursor.fetchall()

columns = {element[0]: list() for element in tables}

for table in tables:

    # Display table name
    print("\n--------------\n")
    print(f"Table Name: {table[0]}")

    # Fetch column names from table
    cursor.execute(f"PRAGMA table_info({table[0]})")
    cols = cursor.fetchall()
    
    # Output column names
    for i, name in enumerate(cols):
        columns[table[0]].append((name[1], name[2]))
        print(f"{i}: {name[1]}, {name[2]}")


--------------

Table Name: Country
0: CountryCode, TEXT
1: ShortName, TEXT
2: TableName, TEXT
3: LongName, TEXT
4: Alpha2Code, TEXT
5: CurrencyUnit, TEXT
6: SpecialNotes, TEXT
7: Region, TEXT
8: IncomeGroup, TEXT
9: Wb2Code, TEXT
10: NationalAccountsBaseYear, TEXT
11: NationalAccountsReferenceYear, TEXT
12: SnaPriceValuation, TEXT
13: LendingCategory, TEXT
14: OtherGroups, TEXT
15: SystemOfNationalAccounts, TEXT
16: AlternativeConversionFactor, TEXT
17: PppSurveyYear, TEXT
18: BalanceOfPaymentsManualInUse, TEXT
19: ExternalDebtReportingStatus, TEXT
20: SystemOfTrade, TEXT
21: GovernmentAccountingConcept, TEXT
22: ImfDataDisseminationStandard, TEXT
23: LatestPopulationCensus, TEXT
24: LatestHouseholdSurvey, TEXT
25: SourceOfMostRecentIncomeAndExpenditureData, TEXT
26: VitalRegistrationComplete, TEXT
27: LatestAgriculturalCensus, TEXT
28: LatestIndustrialData, NUMERIC
29: LatestTradeData, NUMERIC
30: LatestWaterWithdrawalData, NUMERIC

--------------

Table Name: CountryNotes
0: Countr

### Visual Inspection of Database Tables

#### Country Table

In [38]:
query = """
SELECT * FROM Country
"""

df_country = pd.read_sql_query(query, connection)
df_country.head()

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
1,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006
2,DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,...,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001
3,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,,East Asia & Pacific,Upper middle income,AS,...,,,2010,,,Yes,2007,,,
4,ADO,Andorra,Andorra,Principality of Andorra,AD,Euro,,Europe & Central Asia,High income: nonOECD,AD,...,,,2011. Population data compiled from administra...,,,Yes,,,2006,


#### CountryNotes Table

In [39]:
query = """
SELECT * FROM CountryNotes
"""

df_count_notes = pd.read_sql_query(query, connection)

df_count_notes.head()

,Countrycode,Seriescode,Description
0,ABW,EG.EGY.PRIM.PP.KD,Sources: Estimated based on UN Energy Statisti...
1,ABW,EG.ELC.RNEW.ZS,Sources: UN Energy Statistics (2014)
2,ABW,EG.FEC.RNEW.ZS,Sources: UN Energy Statistics (2014)
3,ABW,SM.POP.NETM,Data sources : United Nations World Population...
4,ABW,SM.POP.TOTL,Estimates are derived from data on foreign-bor...


#### Series Table

In [8]:
query = """
SELECT * FROM Series
"""

df_series = pd.read_sql_query(query, connection)

df_series.head()

,SeriesCode,Topic,IndicatorName,ShortDefinition,LongDefinition,UnitOfMeasure,Periodicity,BasePeriod,OtherNotes,AggregationMethod,LimitationsAndExceptions,NotesFromOriginalSource,GeneralComments,Source,StatisticalConceptAndMethodology,DevelopmentRelevance,RelatedSourceLinks,OtherWebLinks,RelatedIndicators,LicenseType
0,BN.KLT.DINV.CD,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net (BoP, current US$)",,Foreign direct investment are the net inflows ...,,Annual,,,,,,Note: Data are based on the sixth edition of t...,"International Monetary Fund, Balance of Paymen...",,,,,,Open
1,BX.KLT.DINV.WD.GD.ZS,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net inflows (% of GDP)",,Foreign direct investment are the net inflows ...,,Annual,,,Weighted average,FDI data do not give a complete picture of int...,,Note: Data starting from 2005 are based on the...,"International Monetary Fund, International Fin...",Data on equity flows are based on balance of p...,Private financial flows - equity and debt - ac...,,,,Open
2,BX.KLT.DINV.CD.WD,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net inflows (BoP, c...",,Foreign direct investment refers to direct inv...,,Annual,,,Sum,FDI data do not give a complete picture of int...,,Note: Data starting from 2005 are based on the...,"International Monetary Fund, Balance of Paymen...",Data on equity flows are based on balance of p...,Private financial flows - equity and debt - ac...,,,,Open
3,BM.KLT.DINV.GD.ZS,Economic Policy & Debt: Balance of payments: C...,"Foreign direct investment, net outflows (% of ...",,Foreign direct investment are the net inflows ...,,Annual,,,Weighted average,,,Note: Data are based on the sixth edition of t...,"International Monetary Fund, International Fin...",,,,,,Open
4,BN.TRF.KOGT.CD,Economic Policy & Debt: Balance of payments: C...,"Net capital account (BoP, current US$)",,Net capital account records acquisitions and d...,,Annual,,,,,,Note: Data are based on the sixth edition of t...,"International Monetary Fund, Balance of Paymen...",,,,,,Open


#### Indicators Table

In [3]:
query = """
SELECT * FROM Indicators
"""

df_indicators = pd.read_sql_query(query, connection)
df_indicators.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


#### SeriesNotes Table

In [42]:
query = """
SELECT * FROM SeriesNotes
"""

df_series_notes = pd.read_sql_query(query, connection)
df_series_notes.head()

,Seriescode,Year,Description
0,SP.ADO.TFRT,YR1960,Interpolated using data for 1957 and 1962.
1,SP.DYN.AMRT.FE,YR1960,"Interpolated using data for 1957 and 1962, if ..."
2,SP.DYN.AMRT.MA,YR1960,"Interpolated using data for 1957 and 1962, if ..."
3,SP.DYN.TO65.FE.ZS,YR1960,Interpolated using data for 1957 and 1962.
4,SP.DYN.TO65.MA.ZS,YR1960,Interpolated using data for 1957 and 1962.


#### Footnotes Table

In [43]:
query = """
SELECT * FROM footnotes
"""

df_footnotes = pd.read_sql_query(query, connection)
df_footnotes.head()

,Countrycode,Seriescode,Year,Description
0,ABW,AG.LND.FRST.K2,YR1990,Not specified
1,ABW,AG.LND.FRST.K2,YR2000,Not specified
2,ABW,AG.LND.FRST.K2,YR2005,Not specified
3,ABW,BX.KLT.DINV.CD.WD,YR1987,Source: United Nations Conference on Trade and...
4,ABW,BX.KLT.DINV.CD.WD,YR1988,Source: United Nations Conference on Trade and...


### Comments

* The CountryNotes, Series, SeriesNotes and Footnotes tables contain superfluous information; the only necessary tables are Country and Indicators.

* The unnecessary tables noted above will be disregarded for both data cleaning and analysis to avoid redundant computation time.

## Data Cleaning

### Find columns that contain missing fields

In [4]:
# dictionary to store missing field columns
missing_data = {element: set() for element in columns.keys()}

# Iterature across each table and column
for table, features in columns.items():
    
    for feat in features:
        # Extract missing fields from specified column and table
        query = f"SELECT * FROM {table} WHERE {feat[0]} IS NULL OR {feat[0]} = ''"
        cursor.execute(query)
        miss = cursor.fetchall()
        # Append column to missing list if containing blanks
        if miss:
            missing_data[table].add(feat[0])

missing_data

{'Country': {'Alpha2Code',
  'AlternativeConversionFactor',
  'BalanceOfPaymentsManualInUse',
  'CurrencyUnit',
  'ExternalDebtReportingStatus',
  'GovernmentAccountingConcept',
  'ImfDataDisseminationStandard',
  'IncomeGroup',
  'LatestAgriculturalCensus',
  'LatestHouseholdSurvey',
  'LatestIndustrialData',
  'LatestPopulationCensus',
  'LatestTradeData',
  'LatestWaterWithdrawalData',
  'LendingCategory',
  'NationalAccountsBaseYear',
  'NationalAccountsReferenceYear',
  'OtherGroups',
  'PppSurveyYear',
  'Region',
  'SnaPriceValuation',
  'SourceOfMostRecentIncomeAndExpenditureData',
  'SpecialNotes',
  'SystemOfNationalAccounts',
  'SystemOfTrade',
  'VitalRegistrationComplete',
  'Wb2Code'},
 'CountryNotes': set(),
 'Series': {'AggregationMethod',
  'BasePeriod',
  'DevelopmentRelevance',
  'GeneralComments',
  'LimitationsAndExceptions',
  'NotesFromOriginalSource',
  'OtherNotes',
  'OtherWebLinks',
  'RelatedIndicators',
  'RelatedSourceLinks',
  'ShortDefinition',
  'Statis

### Fill missing values

1. Confirm all blanks are stored as '' rather than NULL

In [44]:
null_count = 0
for feature in missing_data['Country']:
    query = f"SELECT COUNT(*) FROM Country WHERE {feature} IS NULL"
    nulls = cursor.execute(query).fetchone()[0]
    null_count += nulls

print(f"Number of NULLS: {null_count}")

Number of NULLS: 0


2. Determine which features' most common value (mode) is a blank

In [6]:
missing_minority = []
missing_majority = []

# Iterate over each Country feature
for feature in missing_data['Country']:
    
    # Extract mode
    query = f"SELECT {feature} FROM Country GROUP BY {feature} ORDER BY COUNT(*) DESC LIMIT 1"
    mode = cursor.execute(query).fetchone()[0]

    # Assign feature to correct group (N.B. IncomeGroup and Region are significant features for later analysis so cannot be filled with mode as analysis results would be impacted)
    if mode == '' or feature in ['IncomeGroup', 'Region']:
        missing_majority.append(feature)
    else:
        missing_minority.append(feature)


3. Fill majority-blank features with "UNKNOWN"

In [7]:
# Iterate over majority-blank features
for feature in missing_majority:
    
    # SQL query to fill blanks with "UNKNOWN"
    query = f"UPDATE Country SET {feature} = 'UNKNOWN' WHERE {feature} = ''"
    cursor.execute(query)

    # Output to confirm no blanks fields remain
    query = f"SELECT COUNT(*) FROM Country WHERE {feature} = ''"
    count_miss = cursor.execute(query).fetchone()[0]
    print(f"{count_miss} blank fields: {feature}")

0 blank fields: NationalAccountsReferenceYear
0 blank fields: OtherGroups
0 blank fields: VitalRegistrationComplete
0 blank fields: SourceOfMostRecentIncomeAndExpenditureData
0 blank fields: AlternativeConversionFactor
0 blank fields: SpecialNotes
0 blank fields: Alpha2Code
0 blank fields: LatestWaterWithdrawalData
0 blank fields: LatestAgriculturalCensus
0 blank fields: Region
0 blank fields: LendingCategory
0 blank fields: CurrencyUnit
0 blank fields: LatestHouseholdSurvey
0 blank fields: ExternalDebtReportingStatus
0 blank fields: LatestIndustrialData
0 blank fields: IncomeGroup


4. Fill minority-blank features with the mode

In [8]:
for feature in missing_minority:
    query = f"SELECT {feature} FROM Country GROUP BY {feature} ORDER BY COUNT(*) DESC LIMIT 1"
    mode = cursor.execute(query).fetchone()[0]
    
    query = f"UPDATE Country SET {feature} = '{mode}' WHERE {feature} = ''"
    cursor.execute(query)

    query = f"SELECT COUNT(*) FROM Country WHERE {feature} = ''"
    count_miss = cursor.execute(query).fetchone()[0]
    print(f"{count_miss} blank fields: {feature}")

0 blank fields: LatestTradeData
0 blank fields: SnaPriceValuation
0 blank fields: GovernmentAccountingConcept
0 blank fields: PppSurveyYear
0 blank fields: LatestPopulationCensus
0 blank fields: BalanceOfPaymentsManualInUse
0 blank fields: NationalAccountsBaseYear
0 blank fields: Wb2Code
0 blank fields: SystemOfNationalAccounts
0 blank fields: ImfDataDisseminationStandard
0 blank fields: SystemOfTrade


### Check for duplicate rows in Country and Indicators tables

In [9]:
# Extracts duplicates from Country table
query = "SELECT TableName, CountryCode, COUNT(*) FROM Country GROUP BY TableName, CountryCode HAVING COUNT(*) > 1"
duplicate_country = cursor.execute(query).fetchall()

# Extracts duplicates from Indicators table
query = """
SELECT CountryCode, IndicatorCode, Year, COUNT(*) FROM Indicators 
GROUP BY CountryCode, IndicatorCode, Year HAVING COUNT(*) > 1
"""
duplicate_indicators = cursor.execute(query).fetchall()

# Outputs duplicate counts
print(f"{len(duplicate_country)} duplicates in Country table, {len(duplicate_indicators)} duplicates in Indicators table")

0 duplicates in Country table, 0 duplicates in Indicators table


#### Comment

There are no duplicated rows in either relevant table so analysis can proceed.

In [12]:
# save changes to database and terminate connection
connection.commit()
connection.close()